# Introduction to pytket-braket

This ``BraketBackend`` can be used to interface with multiple simulators and quantum processors. 

Lets use a simple GHZ circuit to demonsrate some features of the ``BraketBackend``.

In [3]:
from pytket import Circuit

def build_ghz_circ(n_qubits: int) -> Circuit:
    """
    Returns a circuit that prepares the n qubit GHZ state.
    """
    circ = Circuit(n_qubits)
    circ.H(0)
    for k in range(1, n_qubits):
        circ.CX(k-1, k)
    
    return circ

In [4]:
circ1 = build_ghz_circ(3) # define a 3 qubit GHZ circuit

The pytket-braket extension offers the ability to convert circuits from a TKET format to a braket format and vice versa.

This is done using the ``tk_to_braket`` and ``braket_to_tk`` functions.

In [5]:
from pytket.extensions.braket import tk_to_braket, braket_to_tk

In [6]:
bk_circ = tk_to_braket(circ1) # convert from a TKET circuit to a braket circuit.
print(bk_circ)

(Circuit('instructions': [Instruction('operator': I('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': I('qubit_count': 1), 'target': QubitSet([Qubit(1)])), Instruction('operator': I('qubit_count': 1), 'target': QubitSet([Qubit(2)])), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)])), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(1), Qubit(2)]))]), {})


In [7]:
from pytket.extensions.braket import BraketBackend # import our BraketBackend

## Braket Backend Parameters

 - **local** – use simulator running on local machine, default: False

 - **local_device** – name of local device (ignored if local=False) – e.g. “braket_sv” (default) or “braket_dm”.

 - **device** – device name from device ARN (e.g. “ionQdevice”, “Aspen-8”, …), default: “sv1”
 
 - **region** - e.g. 'eu-west-2' (for London), 'us-west-1' (for Northern California)

 - **s3_bucket** – name of S3 bucket to store results

 - **s3_folder** – name of folder (“key”) in S3 bucket to store results in

 - **device_type** – device type from device ARN (e.g. “qpu”), default: “quantum-simulator”

 - **provider** – provider name from device ARN (e.g. “ionq”, “rigetti”, “oqc”, …), default: “amazon”

 - **aws_session** – braket AwsSession object, to pass credentials in if not configured on local machine




If we set ``local=True`` then the only available simulator is the statevector simulator SV1. If used locally this can support a maximum of 26 qubits. If instead we use access this simulator remotely it can support up to 34 qubits.

In [8]:
aws_backend_local = BraketBackend(local=True) #initialise local simulator

Lets use a simple GHZ circuit to demonstrate backends available in pytket-braket.

In [9]:
aws_backend_local.backend_info # lets look at the device info and supported operations

BackendInfo(name='BraketBackend', device_name='sv1', version='0.19.0', architecture=<tket::Architecture, nodes=26>, gate_set={<OpType.XXPhase: 66>, <OpType.YYPhase: 67>, <OpType.ZZPhase: 68>, <OpType.ISWAPMax: 73>, <OpType.Z: 19>, <OpType.X: 20>, <OpType.Y: 21>, <OpType.S: 22>, <OpType.Sdg: 23>, <OpType.T: 24>, <OpType.Tdg: 25>, <OpType.V: 26>, <OpType.Vdg: 27>, <OpType.H: 30>, <OpType.Rx: 31>, <OpType.Ry: 32>, <OpType.Rz: 33>, <OpType.U1: 36>, <OpType.CX: 39>, <OpType.CY: 40>, <OpType.CZ: 41>, <OpType.CV: 43>, <OpType.CU1: 50>, <OpType.CCX: 53>, <OpType.SWAP: 54>, <OpType.CSWAP: 55>, <OpType.noop: 57>, <OpType.ECR: 61>, <OpType.ISWAP: 62>}, supports_fast_feedforward=False, supports_reset=False, supports_midcircuit_measurement=False, all_node_gate_errors=None, all_edge_gate_errors=None, all_readout_errors=None, averaged_node_gate_errors=None, averaged_edge_gate_errors=None, averaged_readout_errors=None, misc={})

Lets now compile our three qubit circuit to a form that can be executed on the local statevector simulator.

In [10]:
compiled_circ = aws_backend_local.get_compiled_circuit(circ1) 
handle = aws_backend_local.process_circuit(compiled_circ)
result = aws_backend_local.get_result(handle)

In [11]:
result # view our entire BackendResult Object - our statevector and density matrix will be calculated

BackendResult(q_bits={q[0]: 0, q[1]: 1, q[2]: 2},c_bits={},counts=None,shots=None,state=[1.11022302e-16-0.70710678j 0.00000000e+00+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j
 0.00000000e+00+0.j         0.00000000e+00-0.70710678j],unitary=None,density_matrix=[[0.5+0.00000000e+00j 0. -0.00000000e+00j 0. -0.00000000e+00j
  0. -0.00000000e+00j 0. -0.00000000e+00j 0. -0.00000000e+00j
  0. -0.00000000e+00j 0.5+7.85046229e-17j]
 [0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j]
 [0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j]
 [0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j 0. +0.00000000e+00j
  0. +0.00000000e+00j 0. +0.00000000e+00j]
 [0. +0.00000000e+0

In [12]:
result.get_state() # our statevector - equal to the GHZ state up to a global phase

array([1.11022302e-16-0.70710678j, 0.00000000e+00+0.j        ,
       0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
       0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
       0.00000000e+00+0.j        , 0.00000000e+00-0.70710678j])

If we add some measurements to our circuit and an ``n_shots`` parameter to ``process_circuit`` then we can get shots based results instead. Our result is obtained by randomly sampling the statevector.

In [13]:
compiled_circ.measure_all()
handle2 = aws_backend_local.process_circuit(compiled_circ, n_shots=1000)
result2 = aws_backend_local.get_result(handle2)

In [14]:
result2.get_counts()

Counter({(0, 0, 0): 493, (1, 1, 1): 507})

We see that we have a roughly 50:50 distribution of shots in our two basis states as we would expect from measuring the GHZ state.

Lets now take a look at how to use access remote simulators with ``pytket-braket`` this requires passing additional parameters to the ``BraketBackend`` constructor. We have to specify a region as well as an s3 bucket and folder where our results will be stored.

**Note:** The quantum devices that are available to use will depend on the region specified.

In [15]:
my_region = 'eu-west-2'
my_s3_bucket = 'callum-cq-awsbucket' 
my_s3_folder = 'aws_test/'

We now read in our AWS credentials namely our access key id and secret access key from an external txt file.

In [16]:
def read_aws_credentials(filename :str):
    """
    Reads in access_key_id and secret_access_key from an external txt file. Returns them as a tuple.
    """
    with open(filename, "r") as f:
        access_key_id, secret_access_key = (s.strip() for s in f.readlines())
        return (access_key_id, secret_access_key)

In [17]:
# unpack credentials
#access_key_id, secret_access_key = read_aws_credentials('aws_cred.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'aws_cred.txt'

In [16]:
# explanation of boto3 and AwsSession to go here.

In [17]:
import boto3

my_boto_session = boto3.Session(aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key,
region_name=my_region)

In [18]:
from braket.aws.aws_session import AwsSession

my_aws_session = AwsSession(boto_session=my_boto_session) # initialise an AwsSession

In [19]:
#BraketBackend.available_devices(region='eu-west-2', aws_session=my_aws_session)

In [20]:
my_aws_session.search_devices() # view available devices and simulators in my region

[{'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/tn1',
  'deviceName': 'TN1',
  'deviceStatus': 'ONLINE',
  'deviceType': 'SIMULATOR',
  'providerName': 'Amazon Braket'},
 {'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
  'deviceName': 'SV1',
  'deviceStatus': 'ONLINE',
  'deviceType': 'SIMULATOR',
  'providerName': 'Amazon Braket'},
 {'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/dm1',
  'deviceName': 'dm1',
  'deviceStatus': 'ONLINE',
  'deviceType': 'SIMULATOR',
  'providerName': 'Amazon Braket'},
 {'deviceArn': 'arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy',
  'deviceName': 'Lucy',
  'deviceStatus': 'ONLINE',
  'deviceType': 'QPU',
  'providerName': 'Oxford'}]

Lets initialise the tensor network simulator TN1. This is a remote simulator that can simulate circuits of up to 50 qubits. This simulator uses tensor network contraction instead of the usual method of statevector simulation. This is especially effective for sparse quantum circuits. To read more about how TN1 works read the [documentation](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-tn1).

In [21]:
aws_tn_backend = BraketBackend(local=False,
                                  device='tn1',
                                   s3_bucket=my_s3_bucket,
                                   s3_folder=my_s3_folder,
                                   device_type='quantum-simulator',
                                  provider='amazon',
                                   aws_session=my_aws_session)

In [22]:
ghz_circ = build_ghz_circ(15)
compiled_tn_circ = aws_tn_backend.get_compiled_circuit(ghz_circ)
#handle = aws_tn_backend.process_circuit(compiled_tn_circ)
#result = aws_tn_backend.get_result(handle)
#print(result)

Finally lets demonstrate accessing a real quantum processor with the ``BraketBackend``. We will use the Lucy device from Oxford Quantum Circuits. This is a superconducting device that can run circuits of up to eight qubits.

In [23]:
#initialise our BraketBackend for the OQC device
aws_oqc_backend = BraketBackend(local=False,
                                   region=my_region,
                                  device='Lucy',
                                   s3_bucket=my_s3_bucket,
                                   s3_folder=my_s3_folder,
                                   device_type='qpu',
                                  provider='oqc',
                                   aws_session=my_aws_session)

In [2]:
# define a circuit to compile to the Lucy device
oqc_circ = build_ghz_circ(3)
oqc_circ.measure_all()

NameError: name 'build_ghz_circ' is not defined

In [28]:
compiled_circ = aws_oqc_backend.get_compiled_circuit(oqc_circ)
oqc_handle = aws_oqc_backend.process_circuit(compiled_circ, n_shots=500)
oqc_result = aws_oqc_backend.get_result(oqc_handle) #Validation exception?

ValidationException: An error occurred (ValidationException) when calling the CreateQuantumTask operation: Caller doesn't have access to callum-cq-awsbucket or it doesn't exist.

In [ ]:
oqc_result